# Model Predictive Control in MuJoCo

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mujoco
import mujoco.viewer as viewer
import mediapy as media

## Model MPC

In [3]:
class ModelPredictiveControl(object):
    
    # A,B,C - system matrices
    # f -  prediction horizon
    # v  - control horizon
    # W3 - input weight matrix
    # W4 - prediction weight matrix
    # x0 - initial state of the system
    # desiredControlTrajectoryTotal - total desired control trajectory
    #                               later on, we will take segments of this
    #                               desired state trajectory
    
    def __init__(self,A,B,C,f,v,W3,W4,x0,desiredControlTrajectoryTotal):
        
        # initialize variables
        self.A=A 
        self.B=B
        self.C=C
        self.f=f
        self.v=v
        self.W3=W3 
        self.W4=W4
        self.desiredControlTrajectoryTotal=desiredControlTrajectoryTotal
        
        # dimensions of the matrices
        
        self.n=A.shape[0]
        self.r=C.shape[0]
        self.m=B.shape[1]        
                
               
        # this variable is used to track the current time step k of the controller
        # after every calculation of the control inpu, this variables is incremented for +1 
        self.currentTimeStep=0
        
        # we store the state vectors of the controlled state trajectory
        self.states=[]
        self.states.append(x0)
        
        # we store the computed inputs 
        self.inputs=[]
        
        # we store the output vectors of the controlled state trajectory
        self.outputs=[]
        
        
        # form the lifted system matrices and vectors
        # the gain matrix is used to compute the solution 
        # here we pre-compute it to save computational time
        self.O, self.M, self.gainMatrix = self.formLiftedMatrices()
        
    
    # this function forms the lifted matrices O and M, as well as the 
    # the gain matrix of the control algorithm
    # and returns them 
    def formLiftedMatrices(self):
        f=self.f
        v=self.v
        r=self.r
        n=self.n
        m=self.m
        A=self.A
        B=self.B
        C=self.C
        
        # lifted matrix O
        O=np.zeros(shape=(f*r,n))

        for i in range(f):
            if (i == 0):
                powA=A;
            else:
                powA=np.matmul(powA,A)
            O[i*r:(i+1)*r,:]=np.matmul(C,powA)

        # lifted matrix M        
        M=np.zeros(shape=(f*r,v*m))
    
        for i in range(f):
            # until the control horizon
            if (i<v):
                for j in range(i+1):
                    if (j == 0):
                        powA=np.eye(n,n);
                    else:
                        powA=np.matmul(powA,A)
                    M[i*r:(i+1)*r,(i-j)*m:(i-j+1)*m]=np.matmul(C,np.matmul(powA,B))
            
            # from control horizon until the prediction horizon
            else:
                for j in range(v):
                    # here we form the last entry
                    if j==0:
                        sumLast=np.zeros(shape=(n,n))
                        for s in range(i-v+2):
                            if (s == 0):
                                powA=np.eye(n,n);
                            else:
                                powA=np.matmul(powA,A)
                            sumLast=sumLast+powA
                        M[i*r:(i+1)*r,(v-1)*m:(v)*m]=np.matmul(C,np.matmul(sumLast,B))
                    else:
                        powA=np.matmul(powA,A)
                        M[i*r:(i+1)*r,(v-1-j)*m:(v-j)*m]=np.matmul(C,np.matmul(powA,B))
        
        
        tmp1=np.matmul(M.T,np.matmul(self.W4,M))
        tmp2=np.linalg.inv(tmp1+self.W3)
        gainMatrix=np.matmul(tmp2,np.matmul(M.T,self.W4))
        
        
        return O,M,gainMatrix
    
    # this function propagates the dynamics
    # x_{k+1}=Ax_{k}+Bu_{k}
    
    def propagateDynamics(self,controlInput,state):

        xkp1=np.zeros(shape=(self.n,1))
        yk=np.zeros(shape=(self.r,1))
        xkp1=np.matmul(self.A,state)+np.matmul(self.B,controlInput)
        yk=np.matmul(self.C,state)
        
        return xkp1,yk
        
    # this function computes the control inputs, applies them to the system 
    # by calling the propagateDynamics() function and appends the lists
    # that store the inputs, outputs, states
    def computeControlInputs(self):
                
        # extract the segment of the desired control trajectory
        desiredControlTrajectory=self.desiredControlTrajectoryTotal[self.currentTimeStep:self.currentTimeStep+self.f,:]

        # compute the vector s
        vectorS=desiredControlTrajectory-np.matmul(self.O,self.states[self.currentTimeStep])
       
        # compute the control sequence
        inputSequenceComputed=np.matmul(self.gainMatrix,vectorS)
        inputApplied=np.zeros(shape=(1,1))
        inputApplied[0,0]=inputSequenceComputed[0,0]
        
        # compute the next state and output
        state_kp1,output_k=self.propagateDynamics(inputApplied,self.states[self.currentTimeStep])
        
        # append the lists
        self.states.append(state_kp1)
        self.outputs.append(output_k)
        self.inputs.append(inputApplied)
        # increment the time step
        self.currentTimeStep=self.currentTimeStep+1
        

## Load Model

In [2]:
# load model
xml = "../mujoco_menagerie/universal_robots_ur5e/scene.xml"
model = mujoco.MjModel.from_xml_path(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model)

## Render random noise

In [ ]:
DURATION  = 5   # seconds
FRAMERATE = 60  # Hz

# Make a new camera, move it to a closer distance.
camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(model, camera)
camera.distance = 1

# Initialize the robot pose.
mujoco.mj_resetDataKeyframe(model, data, 1)

frames = []
while data.time < DURATION:
    # Set control vector.
    data.ctrl = np.random.randn(model.nu)
    
    # Step the simulation.
    mujoco.mj_step(model, data)    

    # Render and save frames.
    if len(frames) < data.time * FRAMERATE:
        # # Set the lookat point to the humanoid's center of mass.
        # camera.lookat = data.body('torso').subtree_com
        
        renderer.update_scene(data, camera)
        pixels = renderer.render()
        frames.append(pixels)

# Display video.
media.show_video(frames, fps=FRAMERATE)

## Define model

In [20]:
#TODO:define the desire point using inverse cinematics
#TODO: defin initial setpoint using inverse cinematics

nu = model.nu # number of actuators
nv = model.nv  # number of degree of freedoms.
nsensordata = model.nsensordata

# set initial state and control
mujoco.mj_resetData(model, data)
# data.ctrl = ctrl0
# data.qpos = qpos0

# define the continuous-time system matrices A, B, C
A = np.zeros((2*nv, 2*nv))
B = np.zeros((2*nv, nu))
C = np.zeros((nsensordata, 2*nv))

epsilon = 1e-6
flg_centered = False

mujoco.mjd_transitionFD(model, data, epsilon, flg_centered, A, B, C, None)

print("A =", A)
print("B =", B)
print("C =", C)


# r=1; m=1 # number of inputs and outputs
# n= 4 # state dimension

A = [[ 9.98138009e-01 -4.40100056e-06  2.12414062e-05  2.91835012e-05
  -8.06708504e-06 -1.56176183e-08  1.62760175e-03  4.27512231e-07
   4.58368045e-06  5.84204341e-06 -1.61408471e-06 -3.12352362e-09]
 [ 2.13756370e-06  9.97457299e-01  2.06155035e-03  8.32015967e-06
  -6.44780460e-07  4.98174703e-08  4.27512707e-07  1.49151454e-03
   4.12158177e-04  1.45051689e-06 -1.06835276e-07  9.96348593e-09]
 [ 2.29184029e-05  2.05752059e-03  9.92773073e-01  7.89088730e-05
  -5.84630410e-06  5.34129969e-07  4.58368028e-06  4.12158177e-04
   5.55045713e-04  1.55520647e-05 -1.14546002e-06  1.06826000e-07]
 [ 1.16840868e-04  1.23481215e-05  3.03624725e-04  9.93730662e-01
  -2.92914419e-05  2.72306103e-06  2.33681722e-05  5.80206583e-06
   6.22082576e-05  7.45953026e-04 -5.83969764e-06  5.44612206e-07]
 [-3.22816944e-05 -4.64334033e-07 -2.21518443e-05 -2.92961928e-05
   9.93408114e-01  1.56113473e-08 -6.45633874e-06 -4.27340940e-07
  -4.58184002e-06 -5.83969764e-06  6.81624767e-04  3.12226948e-09]
 